### 라이브러리 import

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

import FinanceDataReader as fdr

# 시각화 사용자 설정
from matplotlib import rcParams
sns.set_style('whitegrid')
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from xgboost import plot_importance as xgb_plot_importance
from lightgbm import plot_importance as lgb_plot_importance

### 함수 정의

In [2]:
import requests
import re
from bs4 import BeautifulSoup

def getStockList() :
    # 인터넷pc 에서는 한국거래소 접속안됨
    #df = fdr.StockListing('KRX')
    
    sosoks = ['0', '1']
    item_code_list = []

    # 코스피, 코스닥 두 장의 종목코드를 읽어온다.
    for sosok in sosoks:
        url_tmpl = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=%s'
        url = url_tmpl % sosok

        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'lxml')

        # 하단의 페이지를 표시하는 부분 중 가장 마지막 페이지의 정보는
        # 'pgRR' 이라는 class 명을 가진 td 태그 안에 있다.
        item_info = soup.find_all('td', {'class':'pgRR'})

        # href 파라미터가 가지고 있는 주소에서 숫자로 되어 있는 값을 
        # 찾아서 저장하면 sosok 값이 [0]에, 페이지 값은 [1]에 들어간다.
        # '[\d]'+ 는 정규식 표현이며, 숫자 값을 계속해서 찾으라는 의미이다.    
        for item in item_info:
            href_addr = item.a.get('href')
            page_info = re.findall('[\d]+', href_addr)
            page = page_info[1]
            page = int(page) + 1

        # 페이지만큼 반복해서 종목코드를 읽어온다.
        for i in range(1, page, 1):
            sub_url = '{}&page={}'.format(url, i)

            page_text = requests.get(sub_url).text
            page_soup = BeautifulSoup(page_text, 'lxml')

            items = page_soup.find_all('a', {'class':'tltle'})
            for item in items:
                item_data = re.search('[\d]+', str(item))
                if item_data:
                    item_code = item_data.group()
                    item_name = item.text
                    result = item_code, item_name
                    item_code_list.append(result)

    df = pd.DataFrame(item_code_list)
    df.columns = ['Code','Name']
    
    return df

In [3]:
# 인자로 주어진 dataframe 에 대해 minmax scale을 적용하여 돌려줍니다
def scale(df, method='minmax') :
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(df)
    output = min_max_scaler.transform(df)
    output = pd.DataFrame(output, columns=df.columns, index=list(df.index.values))
    return output

# 인자로 주어진 dataframe 안에 있는 칼럼들 중, cols (list 형태) 안에 있는 칼럼들에 대해 그래프를 그려줍니다
# 세로선을 표시하고 싶을 경우 axv_list 에 날짜, 색깔(필수값 아님, 기본 값 파랑('b')) 튜플(ex: (2021, 2, 10, 'b'))의 리스트를 넣어주면 됩니다.
def graph(df, cols, axv_list=[]) :
    rcParams['figure.figsize'] = 11.7,8.27
    ax = sns.lineplot(data=df[cols])
    
    plt.title('compare : ' + (', '.join(cols)), fontsize=20)
    plt.ylabel('Value', fontsize=14)
    plt.xlabel('Date', fontsize=14)
    plt.legend(fontsize=12, loc='best')
    
    if axv_list != [] :
        for i in axv_list :
            if len(i) == 4 :
                color = i[3]
            else :
                color = 'b'
            plt.axvline(x=datetime(i[0], i[1], i[2]), color=color, linestyle='--', linewidth=1)

    plt.show()
    return

# 인자로 주어진 dataframe의 모든 칼럼들에 대해 산점도, 상관관계 히트맵, minmax scale 후 2개 값씩 비교 그래프를 그려줍니다
# 그래프를 그릴 때 기준 칼럼은 pivot 인자에 명시하면 됩니다.(기본 값 : 'samsung' 칼럼) 
def corr(df, pivot='samsung', graph=False) :
    rcParams['figure.figsize'] = 11.7,8.27
    sns.pairplot(df, kind='reg')
    plt.show()
    
    sns.heatmap(df.corr(), annot=True, cmap='summer')
    plt.show()
   
    if graph :
        # minmax scale 시킨 후 그래프 그리기, 상관계수 찍기
        df_scaled = scale(df)
        compare_list = df_scaled.columns.tolist()

        for i in compare_list :
            if i == pivot :
                continue
            graph(df_scaled, [compare_list[0], i])
    return

### 기준(삼성전자) 주가 데이터 가져오기 및 전처리

In [4]:
startYear = '2018'
# 삼성전자 주가 가져오기
samsung = fdr.DataReader('005930', startYear)
samsung

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-01-02,51380,51400,50780,51020,169485,0.001177
2018-01-03,52540,52560,51420,51620,200270,0.011760
2018-01-04,52120,52180,50640,51080,233909,-0.010461
2018-01-05,51300,52120,51200,52120,189623,0.020360
2018-01-08,52400,52520,51500,52020,167673,-0.001919
...,...,...,...,...,...,...
2021-02-15,83800,84500,83300,84200,23529706,0.031863
2021-02-16,84500,86000,84200,84900,20483100,0.008314
2021-02-17,83900,84200,83000,83200,18307735,-0.020024


In [5]:
# 삼성전자 주가 중 값이 이상한 부분은 삭제
# 아래 DF INDEX에 해당하는 날짜에는 값이 없다
samsung[(samsung['Open'] <= 0) | (samsung['High'] <= 0) | (samsung['Low'] <= 0)]

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-04-30,0,0,0,53000,0,0.0
2018-05-02,0,0,0,53000,0,0.0
2018-05-03,0,0,0,53000,0,0.0


In [6]:
col = ['Open', 'High', 'Low', 'Volume', 'Change']
date = ['2018-04-30', '2018-05-02', '2018-05-03']
prevDate = '2018-04-27'

for i in date :
    for j in col :
        samsung[i:i][j] = samsung[prevDate:prevDate][j].values[0]

samsung['2018-04-30':'2018-05-03']

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-04-30,53380,53639,52440,53000,606216,0.016494
2018-05-02,53380,53639,52440,53000,606216,0.016494
2018-05-03,53380,53639,52440,53000,606216,0.016494


In [7]:
samsung['test'] = samsung['Close'].pct_change(3)
samsung['test'] = samsung['test'].shift(-3)
samsung

,Open,High,Low,Close,Volume,Change,test
Date,,,,,,,
2018-01-02,51380,51400,50780,51020,169485,0.001177,0.021560
2018-01-03,52540,52560,51420,51620,200270,0.011760,0.007749
2018-01-04,52120,52180,50640,51080,233909,-0.010461,-0.013312
2018-01-05,51300,52120,51200,52120,189623,0.020360,-0.062932
2018-01-08,52400,52520,51500,52020,167673,-0.001919,-0.072664
...,...,...,...,...,...,...,...
2021-02-15,83800,84500,83300,84200,23529706,0.031863,-0.024941
2021-02-16,84500,86000,84200,84900,20483100,0.008314,-0.027091
2021-02-17,83900,84200,83000,83200,18307735,-0.020024,NaN


In [8]:
# 여러 lag 들의 상관관계 측정을 위해 column 추가
# EX: Change_1 은 당일 종가 기준 1일 후의 변화율 값
maxlag = 5
for i in range(1, maxlag+1) :
    samsung['Close_'+str(i)] = samsung.Close.shift((-1)*i)
    samsung['Change_tmp'] = samsung.Close.pct_change(i)
    samsung['Change_'+str(i)] = samsung['Change_tmp'].shift((-1)*i)
    samsung.drop(['Change_tmp'], axis=1, inplace=True)
samsung.dropna(inplace=True, axis=0)
samsung

,Open,High,Low,Close,Volume,Change,test,Close_1,Change_1,Close_2,Change_2,Close_3,Change_3,Close_4,Change_4,Close_5,Change_5
Date,,,,,,,,,,,,,,,,,
2018-01-02,51380,51400,50780,51020,169485,0.001177,0.021560,51620.0,0.011760,51080.0,0.001176,52120.0,0.021560,52020.0,0.019600,50400.0,-0.012152
2018-01-03,52540,52560,51420,51620,200270,0.011760,0.007749,51080.0,-0.010461,52120.0,0.009686,52020.0,0.007749,50400.0,-0.023634,48840.0,-0.053855
2018-01-04,52120,52180,50640,51080,233909,-0.010461,-0.013312,52120.0,0.020360,52020.0,0.018403,50400.0,-0.013312,48840.0,-0.043853,48240.0,-0.055599
2018-01-05,51300,52120,51200,52120,189623,0.020360,-0.062932,52020.0,-0.001919,50400.0,-0.033001,48840.0,-0.062932,48240.0,-0.074444,48200.0,-0.075211
2018-01-08,52400,52520,51500,52020,167673,-0.001919,-0.072664,50400.0,-0.031142,48840.0,-0.061130,48240.0,-0.072664,48200.0,-0.073433,48540.0,-0.066897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-04,83500,83800,82100,82500,24171688,-0.024823,0.002424,83500.0,0.012121,83000.0,0.006061,82700.0,0.002424,81600.0,-0.010909,84200.0,0.020606
2021-02-05,83100,84000,82500,83500,18036835,0.012121,-0.022754,83000.0,-0.005988,82700.0,-0.009581,81600.0,-0.022754,84200.0,0.008383,84900.0,0.016766
2021-02-08,83800,84200,83000,83000,15338765,-0.005988,0.014458,82700.0,-0.003614,81600.0,-0.016867,84200.0,0.014458,84900.0,0.022892,83200.0,0.002410


### 한국 주식 종목들과 기준(삼성전자)의 상관관계 측정

In [9]:
# 모든 종목과 삼성전자의 상관관계 측정
# 종가(비정상성), 변화율(정상성) 대상

#stocks = fdr.StockListing('KRX')
stocks = getStockList()
stocks

,Code,Name
0,005930,삼성전자
1,000660,SK하이닉스
2,051910,LG화학
3,035420,NAVER
4,005935,삼성전자우
...,...,...
3052,323210,이베스트이안스팩1호
3053,347140,케이프이에스제4호
3054,158310,스타모빌리티
3055,021045,대호피앤씨우


In [10]:
#stocks = stocks[['Symbol', 'Name']]
stocks = stocks[['Code', 'Name']]
# 500 건 이상 데이터가 있는 종목을 가져온다
pivot_len = 500

# 각 lag들의 상관관계 측정

# [lag][(code, name, closeCorr, changeCorr)]
corrList = []

for i in range(0, maxlag+1) :
    corrList.append([])
    
for i, s in stocks.iterrows():
    #code = s['Symbol']
    code = s['Code']
    name = s['Name']
    
    comp = pd.DataFrame(index=samsung.index)
    try :
        tmp = fdr.DataReader(code, startYear)
    except :
        continue
    
    if not ('Close' in tmp.columns.tolist()) :
        continue
    
    # 값이 너무 없는건 제외
    if len(tmp) < pivot_len :
        continue
    
    print(f'{name} : {code}')
    
    # 기본적인 상관관계
    comp['samsung'] = samsung.Close
    comp['other'] = tmp.Close
    comp.dropna(inplace=True, axis=0)
    closeCorr = comp.corr()['other']['samsung']

    comp['samsung'] = samsung.Change
    comp['other'] = tmp.Change
    comp.dropna(inplace=True, axis=0)
    changeCorr = comp.corr()['other']['samsung']

    corrList[0].append((code, name, closeCorr, changeCorr))
        
    # lag 별 상관관계    
    for i in range(1, maxlag+1) :
        comp['samsung'] = samsung['Close_'+str(i)]
        comp['other'] = tmp.Close
        comp.dropna(inplace=True, axis=0)
        closeCorr = comp.corr()['other']['samsung']

        comp['samsung'] = samsung['Change_'+str(i)]
        comp['other'] = tmp.Change
        comp.dropna(inplace=True, axis=0)
        changeCorr = comp.corr()['other']['samsung']

        corrList[i].append((code, name, closeCorr, changeCorr))

삼성전자 : 005930
SK하이닉스 : 000660
LG화학 : 051910
NAVER : 035420
삼성전자우 : 005935
삼성SDI : 006400
삼성바이오로직스 : 207940
현대차 : 005380
카카오 : 035720
셀트리온 : 068270
기아차 : 000270
현대모비스 : 012330
LG전자 : 066570
SK이노베이션 : 096770
LG생활건강 : 051900
삼성물산 : 028260
POSCO : 005490
엔씨소프트 : 036570
SK텔레콤 : 017670
SK : 034730
KB금융 : 105560
LG : 003550
신한지주 : 055550
삼성에스디에스 : 018260
한국전력 : 015760
삼성생명 : 032830
삼성전기 : 009150
아모레퍼시픽 : 090430
포스코케미칼 : 003670
넷마블 : 251270
하나금융지주 : 086790
KT&G : 033780
롯데케미칼 : 011170
S-Oil : 010950
한온시스템 : 018880
LG디스플레이 : 034220
삼성화재 : 000810
한화솔루션 : 009830
현대글로비스 : 086280
고려아연 : 010130
금호석유 : 011780
한국조선해양 : 009540
KT : 030200
우리금융지주 : 316140
CJ제일제당 : 097950
미래에셋대우 : 006800
한국타이어앤테크놀로지 : 161390
기업은행 : 024110
KODEX 200 : 069500
LG이노텍 : 011070
LG유플러스 : 032640
HMM : 011200
오리온 : 271560
현대제철 : 004020
한국금융지주 : 071050
강원랜드 : 035250
아모레G : 002790
SKC : 011790
이마트 : 139480
코웨이 : 021240
대한항공 : 003490
현대건설 : 000720
한미사이언스 : 008930
녹십자 : 006280
유한양행 : 000100
SK케미칼 : 285130
두산중공업 : 034020
동서 : 026960
신

디아이 : 003160
KINDEX 베트남VN30(합성) : 245710
노루페인트 : 090350
S-Oil우 : 010955
KBSTAR 코스닥150선물레버리지 : 278240
삼양통상 : 002170
대한제당 : 001790
한국주철관 : 000970
동일산업 : 004890
화신 : 010690
신세계건설 : 034300
DB : 012030
세아제강지주 : 003030
삼성 KTOP30 ETN : 530013
테이팩스 : 055490
AJ네트웍스 : 095570
일동홀딩스 : 000230
인바이오젠 : 101140
하나 코스피 변동성추세 추종 양매도 ETN : 700001
KBSTAR 단기통안채 : 196230
KBSTAR 코스피 : 302450
백광산업 : 001340
태경케미컬 : 006890
미원홀딩스 : 107590
삼일씨엔에스 : 004440
KBSTAR 코스닥150 : 270810
KODEX 반도체 : 091160
이엔플러스 : 074610
TIGER 중국소비테마 : 150460
신풍제약우 : 019175
성창기업지주 : 000180
대창 : 012800
모나리자 : 012690
영풍제지 : 006740
서연 : 007860
국제약품 : 002720
벽산 : 007210
국동 : 005320
한국화장품제조 : 003350
HDC아이콘트롤스 : 039570
한솔홈데코 : 025750
동남합성 : 023450
태경산업 : 015890
ARIRANG 고배당주 : 161510
CJ씨푸드 : 011150
TIGER 단기채권액티브 : 272580
계양전기 : 012200
디와이 : 013570
써니전자 : 004770
카프로 : 006380
신라교역 : 004970
코스맥스비티아이 : 044820
지엠비코리아 : 013870
NH투자증권우 : 005945
STX엔진 : 077970
미래에셋 레버리지 원유선물혼합 ETN(H) : 520009
사조대림 : 003960
대한화섬 : 003830
TIGER 차이나CSI300레버리지(합성) : 204480
깨끗

KBSTAR 글로벌4차산업IT(합성 H) : 276650
KBSTAR 200선물레버리지 : 252400
미래에셋 일본 중형주 ETN(H) : 590006
TIGER 미국나스닥바이오 : 203780
신한 브렌트원유 선물 ETN(H) : 500004
TIGER 코스닥150선물인버스 : 250780
ARIRANG 코스피 : 227830
ARIRANG 미국S&P500(H) : 269540
미래에셋 유럽 중형주 ETN(H) : 590004
파워 200 : 152870
일양약품우 : 007575
미래에셋 호주 ASX200 ETN(H) : 520012
KOSEF 미국달러선물 : 138230
세방우 : 004365
TIGER 200 건설 : 139220
KOSEF 인도Nifty50(합성) : 200250
삼성 미국 대형 가치주 ETN(H) : 530016
신한 금 선물 ETN(H) : 500016
KOSEF 200선물레버리지 : 253250
삼성 미국 대형 가치주 ETN : 530020
TIGER 차이나CSI300인버스(합성) : 217780
에이리츠 : 140910
KODEX 200가치저변동 : 223190
하이골드12호 : 172580
TIGER 200 금융 : 139270
미래에셋 유럽 대형주 ETN(H) : 590003
대상우 : 001685
KODEX 미국S&P고배당커버드콜(합성 H) : 276970
KINDEX 미국WideMoat가치주 : 309230
KODEX 고배당 : 279530
KODEX 배당성장 : 211900
세기상사 : 002420
삼성 인버스 2X WTI원유 선물 ETN : 530036
미래에셋 Q150 Core5 ETN : 520013
KODEX 헬스케어 : 266420
모두투어리츠 : 204210
TRUE 레버리지 유로스탁스50 ETN(H) : 570026
QV 의료 TOP5 ETN : 550012
KOSEF 코스닥150 : 316670
TIGER 화장품 : 228790
KODEX 필수소비재 : 266410
KINDEX 골드선물 레버리지(합성 H

TIGER S&P글로벌인프라(합성) : 269370
KBSTAR 코스닥150선물인버스 : 275750
유유제약2우B : 000227
성문전자우 : 014915
진흥기업우B : 002785
KODEX 밸류Plus : 244670
KINDEX 국채선물10년인버스 : 299070
TIGER 지주회사 : 307520
KODEX 미국채울트라30년선물인버스(H) : 304670
KBSTAR 200선물인버스 : 252410
KOSEF 코스닥150선물 : 291610
ARIRANG 200동일가중 : 295820
KOSEF 코스닥150선물인버스 : 291620
KODEX China H선물인버스(H) : 291660
KBSTAR 200생활소비재 : 287330
TIGER 200 생활소비재 : 227560
TIGER 200 산업재 : 227550
미래에셋 인버스 전기전자 Core5 ETN : 520005
TIGER 원유선물인버스(H) : 217770
동양우 : 001525
ARIRANG ESG우수기업 : 278420
KOSEF 200선물인버스 : 253240
KBSTAR 미국장기국채선물인버스(H) : 267450
TIGER 일본엔선물레버리지 : 292570
삼성 인버스 2X 구리 선물 ETN(H) : 530064
KODEX WTI원유선물인버스(H) : 271050
KBSTAR 수출주 : 140570
KBSTAR 중소형모멘텀로우볼 : 300280
KBSTAR 미국장기국채선물레버리지(합성 H) : 267490
KBSTAR 국채선물10년인버스 : 295020
TIGER 일본엔선물인버스 : 292580
KODEX 골드선물인버스(H) : 280940
TIGER 금속선물(H) : 139310
TIGER 200 철강소재 : 139240
신한 인버스 2X 구리 선물 ETN : 500041
ARIRANG S&P글로벌인프라 : 269530
KBSTAR 모멘텀밸류 : 252720
ARIRANG 코스닥150 : 301400
동양2우B : 001527
KBSTAR 우량업종 : 140580
TIGER 일

와이지-원 : 019210
강스템바이오텍 : 217730
인지디스플레 : 037330
동운아나텍 : 094170
캔서롭 : 180400
코렌텍 : 104540
기가레인 : 049080
큐로컴 : 040350
동국산업 : 005160
JTC : 950170
힘스 : 238490
리드코프 : 012700
금화피에스시 : 036190
오션브릿지 : 241790
유니크 : 011320
에이테크솔루션 : 071670
베스파 : 299910
유니테크노 : 241690
제이씨현시스템 : 033320
엠플러스 : 259630
나라엠앤디 : 051490
푸드나무 : 290720
비씨월드제약 : 200780
이원컴포텍 : 088290
대한뉴팜 : 054670
덱스터 : 206560
우리기술 : 032820
엑셈 : 205100
오르비텍 : 046120
신일제약 : 012790
아이쓰리시스템 : 214430
중앙백신 : 072020
에스엠코어 : 007820
시스웍 : 269620
줌인터넷 : 239340
에이치엔티 : 176440
탑엔지니어링 : 065130
디에이테크놀로지 : 196490
파라텍 : 033540
팬엔터테인먼트 : 068050
러셀 : 217500
덕우전자 : 263600
솔본 : 035610
필로시스헬스케어 : 057880
엑시콘 : 092870
서원인텍 : 093920
바이오리더스 : 142760
리메드 : 302550
SM C&C : 048550
대양전기공업 : 108380
코스맥스엔비티 : 222040
비디아이 : 148140
삼목에스폼 : 018310
스페코 : 013810
아이원스 : 114810
모바일어플라이언스 : 087260
네온테크 : 306620
팜스토리 : 027710
비트컴퓨터 : 032850
파나진 : 046210
덴티스 : 261200
오리엔트정공 : 065500
케어랩스 : 263700
메카로 : 241770
SDN : 099220
비아트론 : 141000
디알젬 : 263690
APS홀딩스 : 054620
국보디자인 : 066620

GRT : 900290
금강철강 : 053260
화신정공 : 126640
지엘팜텍 : 204840
웨이브일렉트로 : 095270
코이즈 : 121850
원바이오젠 : 307280
소프트센 : 032680
체리부로 : 066360
형지I&C : 011080
토박스코리아 : 215480
카스 : 016920
블러썸엠앤씨 : 263920
코렌 : 078650
오하임아이엔티 : 309930
알톤스포츠 : 123750
글로벌에스엠 : 900070
에프알텍 : 073540
좋은사람들 : 033340
대호피앤씨 : 021040
그리티 : 204020
씨유메디칼 : 115480
아스타 : 246720
한일네트웍스 : 046110
화성밸브 : 039610
젬백스지오 : 041590
신원종합개발 : 017000
TPC : 048770
포시에스 : 189690
피제이메탈 : 128660
픽셀플러스 : 087600
아시아종묘 : 154030
캐스텍코리아 : 071850
오스테오닉 : 226400
한일단조 : 024740
네오펙트 : 290660
플랜티넷 : 075130
위즈코프 : 038620
명성티엔에스 : 257370
성우테크론 : 045300
신진에스엠 : 138070
아시아경제 : 127710
에코캡 : 128540
희림 : 037440
얼라인드 : 238120
신라에스지 : 025870
삼일기업공사 : 002290
아세아텍 : 050860
휴네시온 : 290270
인화정공 : 101930
에이팸 : 073070
케이에스피 : 073010
쎄니트 : 037760
정원엔시스 : 045510
핸디소프트 : 220180
조이맥스 : 101730
백금T&A : 046310
나노캠텍 : 091970
알에프세미 : 096610
파루 : 043200
씨엔플러스 : 115530
라이브플렉스 : 050120
에프앤리퍼블릭 : 064090
스맥 : 099440
상상인인더스트리 : 101000
한솔인티큐브 : 070590
아이크래프트 : 052460
COWON : 056000
골든센츄리 : 9

In [11]:
def sortedByCloseCorrPrint(lag=0, n=50) :
    sorted_by_closeCorr = []
    for i in range(0, maxlag+1) :
        # closeCorr 로 정렬
        sorted_by_closeCorr.append(sorted(corrList[i], key=lambda tup: tup[2]))
        
    # closecorr : 값이 제일 작은 순서, 제일 큰 순서대로 n개 출력
    print(f"삼성전자의 {lag}일 후 Close와 상관관계 확인\n")

    closeCorr_len = len(sorted_by_closeCorr[lag])
    print(f"***** Close Corr : 제일 작은것부터 {n}개 *****")
    for i in range (0, n) :
        print(f'{sorted_by_closeCorr[lag][i][1]}({sorted_by_closeCorr[lag][i][0]}) : {sorted_by_closeCorr[lag][i][2]}')
    print()
    print(f"***** Close Corr : 제일 큰것부터 {n}개 *****")
    for i in range (n, 0, -1) :
        print(f'{sorted_by_closeCorr[lag][closeCorr_len-(i+1)][1]}({sorted_by_closeCorr[lag][closeCorr_len-(i+1)][0]}) : {sorted_by_closeCorr[lag][closeCorr_len-(i+1)][2]}')
    print()
    return

def sortedByChangeCorrPrint(lag=0, n=50) :
    sorted_by_changeCorr = []
    for i in range(0, maxlag+1) :
        # closeCorr 로 정렬
        sorted_by_changeCorr.append(sorted(corrList[i], key=lambda tup: tup[3]))

    # changecorr : 값이 제일 작은 순서, 제일 큰 순서대로 n개 출력
    print(f"삼성전자의 {lag}일 후 Change와 상관관계 확인\n")
    
    changeCorr_len = len(sorted_by_changeCorr[lag])
    print(f"***** Change Corr : 제일 작은것부터 {n}개 *****")
    for i in range (0, n) :
        print(f'{sorted_by_changeCorr[lag][i][1]}({sorted_by_changeCorr[lag][i][0]}) : {sorted_by_changeCorr[lag][i][3]}')
    print()
    print(f"***** Change Corr : 제일 큰것부터 {n}개 *****")
    for i in range (n, 0, -1) :
        print(f'{sorted_by_changeCorr[lag][changeCorr_len-(i+1)][1]}({sorted_by_changeCorr[lag][changeCorr_len-(i+1)][0]}) : {sorted_by_changeCorr[lag][changeCorr_len-(i+1)][3]}')
    print()
    return

In [12]:
# 삼성전자 변화율과 다른 종목 변화율의 상관계수 정렬하여 출력
for i in range(0, maxlag+1) :
    sortedByCloseCorrPrint(i, 5)
    sortedByChangeCorrPrint(i, 5)
    print()

삼성전자의 0일 후 Close와 상관관계 확인

***** Close Corr : 제일 작은것부터 5개 *****
미래에셋 인버스 전기전자 Core5 ETN(520005) : -0.943138320088539
HANARO 200선물인버스(306520) : -0.9144554785724629
KBSTAR 200선물인버스(252410) : -0.8882183631937156
KOSEF 200선물인버스(253240) : -0.8836266196305795
TRUE 코스피 커버드풋 ETN B(570021) : -0.8830524071892718

***** Close Corr : 제일 큰것부터 5개 *****
KODEX IT(266370) : 0.9642092361605795
KBSTAR 200IT(285000) : 0.9682407310486748
TIGER 200 IT(139260) : 0.9683342847332111
ARIRANG KRX300IT(309170) : 0.9779819741142997
삼성전자우(005935) : 0.9912605493148269

삼성전자의 0일 후 Change와 상관관계 확인

***** Change Corr : 제일 작은것부터 5개 *****
HANARO 200선물인버스(306520) : -0.854229516774218
TIGER 200선물인버스2X(252710) : -0.827451377787429
KBSTAR 200선물인버스2X(252420) : -0.8268847435193795
ARIRANG 200선물인버스2X(253160) : -0.8261778679761418
TIGER 인버스(123310) : -0.8250400666290674

***** Change Corr : 제일 큰것부터 5개 *****
HANARO 200선물레버리지(304780) : 0.85555570946268
KBSTAR 대형고배당10TR(315960) : 0.864702103806048
TIGER MSCI Korea TR(310970) : 0.87

In [13]:
sortedByCloseCorrPrint(1)
sortedByChangeCorrPrint(1)

삼성전자의 1일 후 Close와 상관관계 확인

***** Close Corr : 제일 작은것부터 50개 *****
미래에셋 인버스 전기전자 Core5 ETN(520005) : -0.9405961638391984
HANARO 200선물인버스(306520) : -0.9127917978535655
KBSTAR 200선물인버스(252410) : -0.8851242159409922
KOSEF 200선물인버스(253240) : -0.8805319749894939
TRUE 코스피 커버드풋 ETN B(570021) : -0.8795038458675987
KODEX 인버스(114800) : -0.877484211709508
KODEX 200선물인버스2X(252670) : -0.8748988003759643
KOSEF 200선물인버스2X(253230) : -0.8739294372640211
TIGER 200선물인버스2X(252710) : -0.8737655184758673
KBSTAR 200선물인버스2X(252420) : -0.8735771064756309
KINDEX 인버스(145670) : -0.8731288975860566
ARIRANG 200선물인버스2X(253160) : -0.8726802666210317
TIGER 인버스(123310) : -0.8708325194901462
신한 코스피 콘도르 4/10% ETN(500039) : -0.8619855520126081
KINDEX 일본TOPIX인버스(합성 H)(205720) : -0.8382488713295884
TIGER 미국S&P500선물인버스(H)(225030) : -0.8371831433812353
미래에셋 인버스 S&P500 ETN(H)(590011) : -0.8341340185614117
TRUE 코스피 양매도 3% OTM ETN(570029) : -0.8284052338720769
신한 인버스 은 선물 ETN(H)(500007) : -0.8274936021001563
신한 인버스 다우존스지수 선물 ETN(H

### Feature 정하기

In [15]:
# 인텔 주가 가져오기
intel = fdr.DataReader('INTC', startYear)
# 마이크론 주가 가져오기
micron = fdr.DataReader('MU', startYear)
# TSMC 주가 : https://kr.investing.com/equities/taiwan-semicon-historical-data
tsmc = pd.read_csv('./tsmc.csv', index_col='Date', parse_dates=True)

# 달러 원화
usdkrw = fdr.DataReader('USD/KRW', startYear) 
# 필라델피아 반도체 지수(SOX)
sox = fdr.DataReader('SOXX', startYear)

# 금 선물
gold = fdr.DataReader('GC', startYear) 
# 코스닥
kosdaq = fdr.DataReader('KQ11', startYear) 
# 코스피
kospi = fdr.DataReader('KS11', startYear) 
'''
kospi50 = fdr.DataReader('KS50', startYear) # 코스피50
kospi100 = fdr.DataReader('KS100', startYear) # 코스피100
kospi200 = fdr.DataReader('KS200', startYear) # 코스피200
'''
# 다우존스
dji = fdr.DataReader('DJI', startYear)
# 나스닥
nasdaq = fdr.DataReader('IXIC', startYear)
# S&P 500
sp500 = fdr.DataReader('US500', startYear)
# VIX
vix = fdr.DataReader('VIX', startYear)
# 항셍(홍콩)
hangseng = fdr.DataReader('HSI', startYear)
# 닛케이 225 선물
jp = fdr.DataReader('JP225', startYear)
# CSI 300 (중국)
csi = fdr.DataReader('CSI300', startYear)
# 비트코인
btc = fdr.DataReader('BTC/KRW', startYear)
# 러셀2000
rutnu = fdr.DataReader('RUTNU', startYear)
# 천연가스 선물
ng = fdr.DataReader('NG', startYear)
# WTI유 선물
cl = fdr.DataReader('CL', startYear)

In [16]:
compare_Change = pd.DataFrame(index = samsung.index)
compare_Change['samsung'] = samsung.Change

#compare_Change['tsmc'] = tsmc.Change
compare_Change['usdkrw'] = usdkrw.Change
#compare_Change['kosdaq'] = kosdaq.Change
#compare_Change['gold'] = gold.Change
#compare_Change['kospi'] = kospi.Change
compare_Change['sox'] = sox.Change

#compare_Change['sp500'] = sp500.Change

#compare_Change['vix'] = vix.Change
#compare_Change['btc'] = btc.Change
#compare_Change['ng'] = ng.Change

# sox 와 상관관계가 높아서 제외
#compare_Change['intel'] = intel.Change
#compare_Change['micron'] = micron.Change

# 다우존스, 나스닥, s&p500은 서로 상관관계가 높아서 s&p500만 포함시킨다
#compare_Change['dji'] = dji.Change
#compare_Change['nasdaq'] = nasdaq.Change

#compare_Change['hangseng'] = hangseng.Change
#compare_Change['jp'] = jp.Change
#compare_Change['csi'] = csi.Change
#compare_Change['cl'] = cl.Change

#compare_Change['rutnu'] = rutnu.Change

In [17]:
pub = 1.0
plb = 0.5
mlb = -1.0
mub = -0.4
predict_lag = 1

# 일치 문자열 (우선)
sameStr = [
    '미래에셋 인버스 전기전자 Core5 ETN',
    '삼성전자우',
    '하나마이크론',
    '래몽래인',
    'DB하이텍',
    '디바이스이엔지',
    'SK하이닉스',
    
    'KODEX 미국채10년선물',
    '미래에셋 인버스 S&P500 ETN(H)',
    'KINDEX S&P아시아TOP50',
    
    '대교',
    '메리츠증권',
    '넥센타이어',
    '오리온',
    '인천도시가스',
    '국전약품',
    
    '삼성 Alerian 에너지인프라 MLP ETN',
    '미래에셋 미국 시니어론 100 ETN'
]
# 포함하지 않는 문자열
excStr = ['200', '코스피', '2X', '삼성그룹']

# (code, name, closeCorr, changeCorr)
changeCorr_len = len(corrList[predict_lag])
i = 0
for i in range(0, changeCorr_len) :
    code = corrList[predict_lag][i][0]
    name = corrList[predict_lag][i][1]
    changeCorr = corrList[predict_lag][i][3]

    # 일치 문자열에 있거나, 상관계수가 일정 범위에 속하면 넣는다 (제외 문자열이 있을 경우 넣지 않는다) 
    if any(x == name for x in sameStr) :
        compare_Change[name] = fdr.DataReader(code, startYear).Change
    elif any(x in name for x in excStr) :
        continue
    
    if ((changeCorr > 0 and changeCorr <= pub and changeCorr >= plb)
        or (changeCorr < 0 and changeCorr >= mlb and changeCorr <= mub)) :
        compare_Change[name] = fdr.DataReader(code, startYear).Change

In [18]:
compare_Change.dropna(inplace=True, axis=0)
print(f'column length : {len(compare_Change.columns)}')
print(compare_Change.columns)
compare_Change

column length : 20
Index(['samsung', 'usdkrw', 'sox', 'SK하이닉스', '삼성전자우', '오리온', 'DB하이텍', '메리츠증권',
       '넥센타이어', '대교', '인천도시가스', '미래에셋 미국 시니어론 100 ETN',
       '삼성 Alerian 에너지인프라 MLP ETN', 'KINDEX S&P아시아TOP50', 'KODEX 미국채10년선물',
       '미래에셋 인버스 S&P500 ETN(H)', '미래에셋 인버스 전기전자 Core5 ETN', '하나마이크론',
       '디바이스이엔지', '국전약품'],
      dtype='object')


,samsung,usdkrw,sox,SK하이닉스,삼성전자우,오리온,DB하이텍,메리츠증권,넥센타이어,대교,인천도시가스,미래에셋 미국 시니어론 100 ETN,삼성 Alerian 에너지인프라 MLP ETN,KINDEX S&P아시아TOP50,KODEX 미국채10년선물,미래에셋 인버스 S&P500 ETN(H),미래에셋 인버스 전기전자 Core5 ETN,하나마이크론,디바이스이엔지,국전약품
Date,,,,,,,,,,,,,,,,,,,,
2018-12-20,-0.011509,-0.0061,-0.0069,-0.028192,-0.029595,-0.016667,-0.031250,0.000000,0.023454,0.002999,0.003328,0.000995,0.010360,-0.011188,-0.001942,0.024343,0.023102,-0.021477,-0.005243,0.002196
2018-12-21,0.000000,0.0056,-0.0170,0.023891,0.009631,-0.004237,-0.004608,-0.004464,0.002083,-0.008969,-0.001658,-0.007455,-0.024729,-0.000539,-0.005837,0.000000,-0.011290,0.013717,-0.002568,-0.002191
2018-12-24,0.003881,-0.0037,-0.0293,0.006667,-0.014308,0.000000,-0.013889,-0.007848,-0.004158,0.001508,-0.001661,-0.002504,-0.016698,0.001617,0.002446,0.025016,-0.013866,-0.004060,-0.017071,0.002196
2018-12-26,-0.011598,-0.0015,0.0600,-0.004967,0.000000,-0.021277,-0.032864,-0.019209,-0.032359,-0.003012,-0.003328,0.000000,-0.042138,-0.001615,0.004880,0.034777,0.017370,-0.008152,0.013370,0.007450
2018-12-27,-0.002608,-0.0032,0.0066,0.024958,0.012903,-0.008696,0.038835,-0.032258,-0.008630,-0.006042,-0.010017,-0.006526,0.043336,-0.001078,-0.008742,-0.053656,-0.022764,0.001370,0.013194,-0.002175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-04,-0.024823,0.0059,0.0082,-0.038462,0.001348,-0.032653,-0.037441,-0.018843,-0.019520,-0.002632,0.000000,0.001796,0.020593,-0.005004,0.003474,0.002227,0.025581,-0.017621,-0.025698,-0.001263
2021-02-05,0.012121,-0.0029,-0.0068,0.020000,0.000000,0.025316,-0.014587,0.021948,0.087289,0.015831,0.002037,0.005379,0.012107,0.012722,0.005193,-0.010000,-0.020408,0.008969,0.063073,-0.020228
2021-02-08,-0.005988,-0.0010,0.0300,-0.019608,-0.004038,0.004115,-0.018092,0.017450,0.063380,0.006494,0.000000,-0.001337,0.001595,-0.005843,-0.007318,-0.006734,0.004630,-0.053333,0.003236,-0.047742


In [19]:
# 파일
# corr_df = compare_Change.corr()
# corr_df = corr_df[((corr_df > 0.7) | (corr_df < -0.7)) & (corr_df != 1)].fillna('-')
# corr_df.to_csv('./corr_df.csv', encoding='euc-kr')
# # Feature 끼리의 상관관계가 있는지 확인 - 특정 칼럼만 포함
# cols1 = []
# corr(compare_Change[cols1])

In [20]:
# 불필요해 보이는 feature 드랍
drop_cols = []
compare_Change_drop = compare_Change.drop(drop_cols, axis=1)

In [21]:
# 최종 dataframe 파일로 저장
compare_Change_drop.to_csv('./df.csv', encoding='euc-kr')